# Processamento de Fake News

### Pré-processamento

In [32]:
import pandas as pd
import numpy as np
import seaborn as sb
import simplejson
import re
from tqdm import tqdm 

In [33]:
data = []
with open('data/fake-news-tera.json') as f:
    for i in range(5000):
        line = next(f)
        data.append(simplejson.loads(line))

df = pd.DataFrame(data)
df.head()

,app,content,date,location_city,location_country,location_lat,location_lng,location_state,profile_avatar,profile_description,...,profile_location,profile_login,profile_name,profile_url,profile_web,related_tweet_content,related_tweet_date,related_tweet_url,source,url
0,Facebook,Veja o passo a passo: http://fb.me/CK6knq9W,2017-10-31T06:00:03.000Z,Brás de Pina,Brasil,-22.832979,-43.297668,Rio de Janeiro,https://pbs.twimg.com/profile_images/339749594...,"Em breve, o melhor GUIA TURÍSTICO do Rio | Soo...",...,Rio de Janeiro (BR),amigonorio,Amigo no Rio,https://twitter.com/amigonorio,http://www.amigonorio.com.br,None,None,None,0,https://twitter.com/608352107/statuses/9252409...
1,Twitter for iPhone,@maika_1020 BAND MAIDのchoose me(｡>人<),2017-11-01T05:59:13.000Z,None,None,NaN,NaN,None,https://pbs.twimg.com/profile_images/920977346...,MOSHIMO SILENT SIREN sevenoops,...,None,msyn_711,まっしゃん,https://twitter.com/msyn_711,None,None,None,None,0,https://twitter.com/621782721/statuses/9256031...
2,Twitter Web Client,"É DONO DO STF. ONDE TODOS SÃO COVARDES, CONFOR...",2017-11-01T06:00:04.000Z,None,Brasil,-10.000000,-55.000000,None,https://pbs.twimg.com/profile_images/919944565...,"QUANDO OS QUE COMANDAM PERDEM A VERGONHA,OS CO...",...,Brasil,CarlitoMoraes,Carlito Moraes,https://twitter.com/CarlitoMoraes,None,Absurdo ... Gilmar se sente o dono do Brasil ....,2017-10-31T14:19:45.000Z,https://twitter.com/887707008/statuses/9253666...,0,https://twitter.com/346220548/statuses/9256033...
3,Twitter Lite,porra a globo corta se fodrrrr,2017-11-03T05:59:52.000Z,Santos,Brasil,-23.856630,-46.270550,São Paulo,https://pbs.twimg.com/profile_images/925930977...,musa de Alceu Valença•\n@justinbieber💙,...,baixada santista,ingcarvalh0,dindi,https://twitter.com/ingcarvalh0,https://www.instagram.com/morena7ropicana/?hl=...,None,None,None,0,https://twitter.com/1315339214/statuses/926328...
4,Twitter Lite,@UOL Deviam ter deixado mata-lo,2017-11-05T05:58:10.000Z,None,None,NaN,NaN,None,https://pbs.twimg.com/profile_images/905322029...,enfrentando novos desafios e vivendo um dia de...,...,None,wall2013,wall2013,https://twitter.com/wall2013,http://walterdesouza2012.blogspot.com.br/,None,None,None,0,https://twitter.com/1172898380/statuses/927052...


In [34]:
df = df.drop(['app', 'location_city', 'location_country', 'location_lat', 'location_lng', 'location_state', 'profile_avatar', 'profile_location', 'profile_name', 'profile_web', 'related_tweet_date', 'source', 'url'], axis=1)
df.head()

,content,date,profile_description,profile_followers,profile_following,profile_login,profile_url,related_tweet_content,related_tweet_url
0,Veja o passo a passo: http://fb.me/CK6knq9W,2017-10-31T06:00:03.000Z,"Em breve, o melhor GUIA TURÍSTICO do Rio | Soo...",18.0,15.0,amigonorio,https://twitter.com/amigonorio,None,None
1,@maika_1020 BAND MAIDのchoose me(｡>人<),2017-11-01T05:59:13.000Z,MOSHIMO SILENT SIREN sevenoops,695.0,674.0,msyn_711,https://twitter.com/msyn_711,None,None
2,"É DONO DO STF. ONDE TODOS SÃO COVARDES, CONFOR...",2017-11-01T06:00:04.000Z,"QUANDO OS QUE COMANDAM PERDEM A VERGONHA,OS CO...",5694.0,4225.0,CarlitoMoraes,https://twitter.com/CarlitoMoraes,Absurdo ... Gilmar se sente o dono do Brasil ....,https://twitter.com/887707008/statuses/9253666...
3,porra a globo corta se fodrrrr,2017-11-03T05:59:52.000Z,musa de Alceu Valença•\n@justinbieber💙,2641.0,1098.0,ingcarvalh0,https://twitter.com/ingcarvalh0,None,None
4,@UOL Deviam ter deixado mata-lo,2017-11-05T05:58:10.000Z,enfrentando novos desafios e vivendo um dia de...,159.0,953.0,wall2013,https://twitter.com/wall2013,None,None


In [37]:
df.shape

(5000, 9)

In [47]:
def find_url(raw):
    if raw == None:
        return None
    txt = re.search('(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', raw)
    if txt:
        return txt.group()
    else:
        return None

print(find_url('oi oi oi oi oi https://docs.python.org/2/library/re.html'))

https://docs.python.org/2/library/re.html


In [71]:
df['URL'] = df['content']
contador = 0 
for i in tqdm(range(5000)):
    cont = find_url(df['content'][i])
    re_cont = find_url(df['related_tweet_content'][i])
    if find_url(str(re_cont)) != None:
        df['URL'][i] = re_cont
    elif cont != None:
        df['URL'][i] = cont
    else:
        df['URL'][i] = None

print(contador)

  0%|          | 0/5000 [00:00<?, ?it/s]/home/marcello/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/marcello/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/home/marcello/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-c

0


In [44]:
df.head()

,content,date,profile_description,profile_followers,profile_following,profile_login,profile_url,related_tweet_content,related_tweet_url,URL
0,Veja o passo a passo: http://fb.me/CK6knq9W,2017-10-31T06:00:03.000Z,"Em breve, o melhor GUIA TURÍSTICO do Rio | Soo...",18.0,15.0,amigonorio,https://twitter.com/amigonorio,None,None,http://fb.me/CK6knq9W
1,@maika_1020 BAND MAIDのchoose me(｡>人<),2017-11-01T05:59:13.000Z,MOSHIMO SILENT SIREN sevenoops,695.0,674.0,msyn_711,https://twitter.com/msyn_711,None,None,None
2,"É DONO DO STF. ONDE TODOS SÃO COVARDES, CONFOR...",2017-11-01T06:00:04.000Z,"QUANDO OS QUE COMANDAM PERDEM A VERGONHA,OS CO...",5694.0,4225.0,CarlitoMoraes,https://twitter.com/CarlitoMoraes,Absurdo ... Gilmar se sente o dono do Brasil ....,https://twitter.com/887707008/statuses/9253666...,https://twitter.com/J__Guerreiro/status/925366...
3,porra a globo corta se fodrrrr,2017-11-03T05:59:52.000Z,musa de Alceu Valença•\n@justinbieber💙,2641.0,1098.0,ingcarvalh0,https://twitter.com/ingcarvalh0,None,None,None
4,@UOL Deviam ter deixado mata-lo,2017-11-05T05:58:10.000Z,enfrentando novos desafios e vivendo um dia de...,159.0,953.0,wall2013,https://twitter.com/wall2013,None,None,None


In [78]:
from goose3 import Goose

In [92]:
df['Article'] = None
for i in tqdm(range(5000)):
    g = Goose()
    url = df['URL'][i]
    print(url)
    if url == None:
        df['Article'][i] = None
    else:
        article = g.extract(url=url)
        print(article.cleaned_text)
        df['Article'][i] = article.cleaned_text

  0%|          | 0/2 [00:00<?, ?it/s]

http://fb.me/CK6knq9W


/home/marcello/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
 50%|█████     | 1/2 [00:07<00:07,  7.27s/it]/home/marcello/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
100%|██████████| 2/2 [00:07<00:00,  3.66s/it]

O WhatsApp começou a liberar há alguns dias a ferramenta que permite apagar uma mensagem enviada para um amigo. O recurso promete ser útil para quando o usuário encaminha algum conteúdo errado ou se arrepende de ter dito algo logo após o envio. A novidade funcionará tanto na versão para Android e iPhone.

Por enquanto, é necessário estar usando a versão beta do WhatsApp a partir da numeração 2.17.400 no sistema do Google. Ainda assim, a função está sendo liberada gradualmente e pode demorar a chegar a todos os celulares. Confira neste tutorial do Olhar Digital como apagar uma mensagem no WhatsApp.

Ao enviar uma mensagem no WhatsApp, o usuário terá um tempo máximo de sete minutos para apaga-la no celular do destinatário. Após isso, o texto ou imagem sumirá apenas no seu celular, como o recurso funcionava anteriormente. Além disso, todos os participantes da conversa serão notificados de que a mensagem foi apagada.

Um outro detalhe importante é que o celular do seu contato precisa estar

In [93]:
df.head()

,content,date,profile_description,profile_followers,profile_following,profile_login,profile_url,related_tweet_content,related_tweet_url,URL,Article
0,Veja o passo a passo: http://fb.me/CK6knq9W,2017-10-31T06:00:03.000Z,"Em breve, o melhor GUIA TURÍSTICO do Rio | Soo...",18.0,15.0,amigonorio,https://twitter.com/amigonorio,None,None,http://fb.me/CK6knq9W,O WhatsApp começou a liberar há alguns dias a ...
1,@maika_1020 BAND MAIDのchoose me(｡>人<),2017-11-01T05:59:13.000Z,MOSHIMO SILENT SIREN sevenoops,695.0,674.0,msyn_711,https://twitter.com/msyn_711,None,None,None,None
2,"É DONO DO STF. ONDE TODOS SÃO COVARDES, CONFOR...",2017-11-01T06:00:04.000Z,"QUANDO OS QUE COMANDAM PERDEM A VERGONHA,OS CO...",5694.0,4225.0,CarlitoMoraes,https://twitter.com/CarlitoMoraes,Absurdo ... Gilmar se sente o dono do Brasil ....,https://twitter.com/887707008/statuses/9253666...,http://ow.ly/boor30gfBli,None
3,porra a globo corta se fodrrrr,2017-11-03T05:59:52.000Z,musa de Alceu Valença•\n@justinbieber💙,2641.0,1098.0,ingcarvalh0,https://twitter.com/ingcarvalh0,None,None,None,None
4,@UOL Deviam ter deixado mata-lo,2017-11-05T05:58:10.000Z,enfrentando novos desafios e vivendo um dia de...,159.0,953.0,wall2013,https://twitter.com/wall2013,None,None,None,None
